# Introdução

Explicar como automatização de processos pode ser usado para lembredas, eventos, coleta de informações e etc.

Por exemplo, mensagens automaticas no Wpp, coleta de dados em bolsa no mercado financeira, emails lembrando de tarefas (como o nosso proprio Moodle da PUC), fazer beckup diaria em banco de dados e etc


A seção Comunicação-Automatizada aborda como a automatização de processos pode ser aplicada para facilitar o envio e o agendamento de mensagens, e-mails e lembretes, eliminando a necessidade de intervenção manual constante. Essa aplicação é especialmente útil em tarefas repetitivas que envolvem comunicação recorrente, como o envio diário de relatórios, confirmações automáticas, lembretes de reuniões ou notificações baseadas em eventos. Ao programar essas tarefas com antecedência, é possível garantir eficiência, reduzir erros humanos e otimizar o tempo de trabalho. Dessa forma, a comunicação automatizada se torna uma ferramenta fundamental dentro do escopo de processos automatizados, mostrando como a integração entre código e rotina pode transformar a maneira como interagimos com fluxos de informação.

In [ ]:
import imaplib
import email
from email.header import decode_header
from email.utils import parsedate_to_datetime
import pywhatkit
import time
import os
from dotenv import load_dotenv

# Carrega variáveis do arquivo .env
load_dotenv()

EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
WHATSAPP_NUMBER = os.getenv("WHATSAPP_NUMBER")

# Conecta ao Gmail via IMAP
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

# Seleciona apenas a caixa de entrada (Inbox)
mail.select("INBOX")

# Busca por e-mails não lidos (não lidos e na inbox)
status, messages = mail.search(None, '(UNSEEN)')
ids = messages[0].split()

if not ids:
    print("Nenhum e-mail novo.")
else:
    # Pega os últimos 3 e-mails não lidos
    ultimos_emails = ids[-3:]

    mensagens = []

    for email_id in ultimos_emails:
        # Recupera o conteúdo do e-mail
        _, msg_data = mail.fetch(email_id, '(RFC822)')
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Extrai o remetente
        from_header = msg['From']
        decoded_from, charset = decode_header(from_header)[0]
        remetente = decoded_from.decode(charset or 'utf-8') if isinstance(decoded_from, bytes) else decoded_from

        # Extrai o subject (assunto)
        subject_header = msg['Subject']
        if subject_header:
            decoded_subject, charset = decode_header(subject_header)[0]
            assunto = decoded_subject.decode(charset or 'utf-8') if isinstance(decoded_subject, bytes) else decoded_subject
        else:
            assunto = "(Sem assunto)"

        # Extrai e formata a data e hora do e-mail
        try:
            data_email = parsedate_to_datetime(msg["Date"])
            data_formatada = data_email.strftime("%d/%m/%Y às %H:%M")
        except:
            data_formatada = "(Data não disponível)"

        mensagens.append(f"📅 {data_formatada}\nDe: {remetente}\nAssunto: {assunto}")

    # Monta a mensagem final
    corpo = "📬 Você recebeu novos e-mails:\n\n" + "\n\n".join(mensagens)

    print("Enviando mensagem no WhatsApp...\n", corpo)

    # Envia tudo em uma única mensagem
    pywhatkit.sendwhatmsg_instantly(WHATSAPP_NUMBER, corpo, wait_time=10)

mail.logout()


## Explicando o Codigo

Este código tem como principal função monitorar a caixa de entrada do e-mail configurado e notificar, via WhatsApp, os três últimos e-mails não lidos, junto com a data, horário, remetente e assunto. Ele começa carregando as credenciais salvas no arquivo `.env`, conectando-se à conta de e-mail via protocolo IMAP. Em seguida, acessa a caixa de entrada, busca por e-mails não lidos e seleciona os três mais recentes. Para cada um deles, extrai o remetente, o assunto e a data/hora de envio. Com essas informações, monta uma mensagem formatada e envia tudo automaticamente para o número de WhatsApp fornecido, utilizando a biblioteca `pywhatkit`. Por fim, o script realiza logout da conta de e-mail, encerrando a conexão de forma segura.

A lógica por trás desse código representa um excelente exemplo de automatização de processos aplicada à comunicação e ao monitoramento de informações. Em vez de abrir manualmente a caixa de entrada e verificar novos e-mails, o usuário recebe uma notificação no WhatsApp com os dados relevantes assim que o código é executado. Essa automação é útil em ambientes onde o tempo de resposta a mensagens é crítico, como no suporte ao cliente, em operações logísticas ou no gerenciamento de múltiplas contas. O código elimina etapas manuais e garante agilidade no acesso à informação, traduzindo-se em produtividade e eficiência na rotina digital.

![WPP automatizado](mensagem_wpp.png)


In [30]:
import imaplib
import smtplib
import email
from email.header import decode_header
from email.message import EmailMessage
from email.utils import parsedate_to_datetime
import os
from dotenv import load_dotenv

# === Carrega variáveis de ambiente ===
load_dotenv()
EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")  # edgarallan0080@gmail.com
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")  # senha de app
TARGET_SENDER = "superedbill@gmail.com"
REPLY_TO = "edgarvideogame@gmail.com"

# === Conecta ao Gmail via IMAP ===
imap = imaplib.IMAP4_SSL("imap.gmail.com")
imap.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
imap.select("INBOX")

# === Busca por e-mails não lidos do remetente específico ===
status, messages = imap.search(None, f'(UNSEEN FROM "{TARGET_SENDER}")')
ids = messages[0].split()

if not ids:
    print("Nenhum e-mail novo do remetente alvo.")
else:
    for num in ids:
        _, msg_data = imap.fetch(num, "(RFC822)")
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Extrai o assunto
        subject_header = msg["Subject"]
        decoded_subject, charset = decode_header(subject_header)[0]
        subject = decoded_subject.decode(charset or 'utf-8') if isinstance(decoded_subject, bytes) else decoded_subject

        # Extrai o corpo
        corpo = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    corpo = part.get_payload(decode=True).decode()
                    break
        else:
            corpo = msg.get_payload(decode=True).decode()

        print("Email recebido de superedbill. Respondendo automaticamente...")

        # === Envia o e-mail para outro destino via SMTP ===
        resposta = EmailMessage()
        resposta["From"] = EMAIL_ADDRESS
        resposta["To"] = REPLY_TO
        resposta["Subject"] = f"[Auto-Forward] {subject}"
        resposta.set_content(f"Este e-mail foi encaminhado automaticamente:\n\n{corpo}")

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            smtp.send_message(resposta)

        print(f"E-mail encaminhado para {REPLY_TO} com sucesso.")

imap.logout()


Email recebido de superedbill. Respondendo automaticamente...
E-mail encaminhado para edgarvideogame@gmail.com com sucesso.


('BYE', [b'LOGOUT Requested'])

## Explicando o Codigo

O código apresentado realiza a automação do monitoramento da caixa de entrada de e-mails por meio do protocolo IMAP. Ele utiliza bibliotecas como `imaplib` para acessar e-mails recebidos e `smtplib` para enviar mensagens de resposta. O script começa carregando variáveis sensíveis de um arquivo `.env`, como o e-mail e senha de aplicativo. Em seguida, conecta-se ao servidor do Gmail e busca por mensagens não lidas de um remetente específico. Se um novo e-mail for identificado, o código extrai seu conteúdo — incluindo remetente, assunto e corpo — e monta uma nova mensagem, que é automaticamente encaminhada a outro destinatário pré-definido. O fluxo se encerra com a confirmação do envio e o logout da sessão de e-mail.

A intuição por trás desse código está diretamente relacionada à automatização de processos rotineiros de comunicação. Em contextos corporativos ou organizacionais, responder ou encaminhar e-mails manualmente pode ser uma tarefa repetitiva, especialmente em fluxos padronizados ou de triagem. Automatizar esse tipo de resposta ou redirecionamento elimina a necessidade de intervenção humana constante, reduz atrasos e garante eficiência operacional. Este código simula o comportamento de um assistente virtual de e-mails, capaz de monitorar, filtrar e executar ações automáticas, representando na prática um exemplo concreto de Robotic Process Automation (RPA) aplicado ao gerenciamento de informação.



![Email automatizado](mensagem_automatica.png)


In [ ]:
import imaplib
import smtplib
import email
from email.header import decode_header
from email.message import EmailMessage
from email.utils import parsedate_to_datetime
import os
import time
from dotenv import load_dotenv

# === Carrega variáveis de ambiente ===
load_dotenv()
EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
TARGET_SENDER = "superedbill@gmail.com"
REPLY_TO = "edgarvideogame@gmail.com"

print("Monitorando caixa de entrada por e-mails de superedbill@gmail.com...")

while True:
    # Conecta ao Gmail via IMAP
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    imap.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    imap.select("INBOX")

    # Busca e-mails não lidos do remetente alvo
    status, messages = imap.search(None, f'(UNSEEN FROM "{TARGET_SENDER}")')
    ids = messages[0].split()

    if ids:
        for num in ids:
            _, msg_data = imap.fetch(num, "(RFC822)")
            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email)

            # Extrai o assunto
            subject_header = msg["Subject"]
            decoded_subject, charset = decode_header(subject_header)[0]
            subject = decoded_subject.decode(charset or 'utf-8') if isinstance(decoded_subject, bytes) else decoded_subject

            # Extrai o corpo
            corpo = ""
            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        corpo = part.get_payload(decode=True).decode()
                        break
            else:
                corpo = msg.get_payload(decode=True).decode()

            print("📨 E-mail detectado de superedbill. Encaminhando resposta...")

            # Envia resposta
            resposta = EmailMessage()
            resposta["From"] = EMAIL_ADDRESS
            resposta["To"] = REPLY_TO
            resposta["Subject"] = f"[Auto-Forward] {subject}"
            resposta.set_content(f"Este e-mail foi encaminhado automaticamente:\n\n{corpo}")

            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
                smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
                smtp.send_message(resposta)


            print(f"✅ E-mail encaminhado com sucesso para {REPLY_TO}!")
            imap.logout()
            break  # ← em vez de exit()

    else:
        print("⏳ Nenhum e-mail novo de superedbill ainda... aguardando.")
        imap.logout()
        time.sleep(20)


Monitorando caixa de entrada por e-mails de superedbill@gmail.com...
⏳ Nenhum e-mail novo de superedbill ainda... aguardando.
⏳ Nenhum e-mail novo de superedbill ainda... aguardando.
📨 E-mail detectado de superedbill. Encaminhando resposta...
✅ E-mail encaminhado com sucesso para edgarvideogame@gmail.com!
⏳ Nenhum e-mail novo de superedbill ainda... aguardando.


KeyboardInterrupt: 

## Explicando o codigo

Este código representa um exemplo claro de automatização contínua de comunicação, operando em loop infinito para monitorar a chegada de novos e-mails de um remetente específico. Ao identificar uma nova mensagem de `superedbill@gmail.com`, ele responde automaticamente encaminhando o conteúdo para outro endereço pré-configurado. Essa lógica simula, no mundo real, a atuação de um assistente virtual que filtra e redireciona informações sem intervenção humana. Em ambientes corporativos, esse tipo de solução pode ser fundamental para garantir que comunicações urgentes sejam rapidamente encaminhadas a setores ou pessoas relevantes, otimizando o fluxo de informação e reduzindo atrasos em decisões críticas.


![WPP automatizado](mensagem_continua.png)